# Library

In [1]:
import pandas as pd
import numpy as np
import re
import warnings

from konlpy.tag import Mecab
from gensim.models import FastText
from nltk.tokenize import sent_tokenize
from sqlalchemy import create_engine
from tqdm import tqdm_notebook

In [2]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 경고무시를 위한 코드
warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
case_list_사기 = pd.read_csv('./00. Data/case_list_사기.csv')

## SBert

### 모델 로드

In [5]:
with tf.device("/device:GPU:0"):
    sbert = SentenceTransformer('jhgan/ko-sbert-sts')

In [6]:
sentence_tokened = []

for x in tqdm_notebook(case_list_사기['X']):
    sentence_tokened.append(sent_tokenize(x))

  0%|          | 0/2262 [00:00<?, ?it/s]

In [ ]:
embeddings_sbert = []

for sentences in tqdm_notebook(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = sbert.encode(sentence)
        else:
            embedding += sbert.encode(sentence)
        count += 1
    embeddings_sbert.append(embedding/count)

### 사용자 입력 임베딩

In [ ]:
from hanja_hangul.src import hanjahangul
# 사용자 입력 전처리
user_input = "저는 3년 전부터 甲에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 1,500만원을 빌려주었습니다. \
    甲은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데, \
    이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다. 이 경우 甲을 사기죄로 처벌할 수 있는지요?"

user_input = hanjahangul.hanja_to_hangul_dueum(user_input)
clear_user_input = re.sub("[^가-힣. ]", '', user_input)

# 사용자 입력 문장 토큰화
user_sentence_tokened = sent_tokenize(clear_user_input)
user_sentence_tokened

['저는 년 전부터 갑에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 만원을 빌려주었습니다.',
 '갑은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데     이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다.',
 '이 경우 갑을 사기죄로 처벌할 수 있는지요']

In [ ]:
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = sbert.encode(sentence)
    else:
        embedding += sbert.encode(sentence)
    count += 1

s_embedding = embedding / count

### 코사인 유사도 계산

In [ ]:
s_similarity = cosine_similarity([s_embedding], embeddings_sbert)
s_similarity = list(enumerate(s_similarity[0]))
s_similarity = sorted(s_similarity, key=lambda x : x[1], reverse = True)

In [ ]:
precidents = []

for i in range(5):
    idx = s_similarity[i][0]
    sim = round(s_similarity[i][1] * 100, 2)

    precidents.append([case_list_사기['판례일련번호'][idx], case_list_사기['사건명'][idx], case_list_사기['사건종류명'][idx], case_list_사기['판례내용'][idx], sim])

In [ ]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
f_result